In [4]:
# Data Structures
import numpy  as np
import pandas as pd
import json

# Corpus Processing
import string
import re
import csv
import nltk.corpus
from unidecode                        import unidecode
from nltk.tokenize                    import word_tokenize
from nltk                             import SnowballStemmer
from nltk.corpus                      import wordnet as wn
from nltk.stem                        import WordNetLemmatizer
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.preprocessing            import normalize
from collections                      import Counter

# K-Means
from sklearn import cluster
from sklearn.cluster import KMeans

# Visualization and Analysis
import matplotlib.pyplot  as plt
import matplotlib.cm      as cm
import seaborn            as sns
from sklearn.datasets                 import make_blobs
from sklearn.metrics                  import silhouette_samples, silhouette_score
from wordcloud                        import WordCloud
from jupyterthemes                    import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False) 

In [2]:
Data = pd.read_csv('DsDnsPrScTch.csv') 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# removes a list of words (ie. stopwords) from a tokenized list.
def removeWords(listOfTokens, listOfWords):
    return [token for token in listOfTokens if token not in listOfWords]

# applies stemming to a list of tokenized words
def applyStemming(listOfTokens, stemmer):
    return [stemmer.stem(token) for token in listOfTokens]

# applies lematization to a list of tokenized words
# Annotate text tokens with POS tags
wnl = WordNetLemmatizer()
def pos_tag_text(tokens):
    def penn_to_wn_tags(pos_tag):
        if pos_tag.startswith('J'):
            return wn.ADJ
        elif pos_tag.startswith('V'):
            return wn.VERB
        elif pos_tag.startswith('N'):
            return wn.NOUN
        elif pos_tag.startswith('R'):
            return wn.ADV
        else:
            return None
        
    tagged_text = nltk.pos_tag(tokens)

    tagged_text_set = [(word.lower(), penn_to_wn_tags(pos_tag))
                         for word, pos_tag in
                         tagged_text]
    return tagged_text_set
# lemmatize text based on POS tags    
def lemmatize_text(tokens):
    
    pos_tagged_text = pos_tag_text(tokens)
    lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag
                         else word                     
                         for word, pos_tag in pos_tagged_text]
    return lemmatized_tokens

# remove punctations
def removePunctuations(listOfTokens):
    test_joined=' '.join(listOfTokens)
    test_punc_removed = [char for char in test_joined if char not in string.punctuation]
    test_punc_removed_join = ''.join(test_punc_removed)
    return [word for word in test_punc_removed_join.split()]

# removes any words composed of less than 2 or more than 21 letters
def twoLetters(listOfTokens):
    twoLetterWord = []
    for token in listOfTokens:
        if len(token) <= 2 or len(token) >= 21:
            twoLetterWord.append(token)
    return twoLetterWord

In [6]:
def processCorpus(corpus, language):   
    stopwords = nltk.corpus.stopwords.words(language)
    #param_stemmer = SnowballStemmer(language)
    other_words = [line.rstrip('\n') for line in open('stopwords_scrapmaker.txt')] # Load .txt file line by line
    
    for document in corpus:
        index = corpus.index(document)
        corpus[index] = corpus[index].replace(u'\ufffd', '8')   # Replaces the ASCII '�' symbol with '8'
        corpus[index] = corpus[index].replace(',', '')          # Removes commas
        corpus[index] = corpus[index].rstrip('\n')              # Removes line breaks
        corpus[index] = corpus[index].casefold()                # Makes all letters lowercase
        
        corpus[index] = re.sub('\W_',' ', corpus[index])        # removes specials characters and leaves only words
        corpus[index] = re.sub("\S*\d\S*"," ", corpus[index])   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
        corpus[index] = re.sub("\S*@\S*\s?"," ", corpus[index]) # removes emails and mentions (words with @)
        corpus[index] = re.sub(r'http\S+', '', corpus[index])   # removes URLs with http
        corpus[index] = re.sub(r'www\S+', '', corpus[index])    # removes URLs with www

        listOfTokens = word_tokenize(corpus[index])
        twoLetterWord = twoLetters(listOfTokens)

        listOfTokens = removeWords(listOfTokens, stopwords)
        listOfTokens = removeWords(listOfTokens, twoLetterWord)
        listOfTokens = removeWords(listOfTokens, other_words)
        
        listOfTokens = removePunctuations(listOfTokens)
        
        #listOfTokens = applyStemming(listOfTokens, param_stemmer)
        listOfTokens = lemmatize_text(listOfTokens)
        listOfTokens = removeWords(listOfTokens, other_words)
        
        corpus[index]   = " ".join(listOfTokens)
        corpus[index] = unidecode(corpus[index])

    return corpus

In [7]:
tweets_df = Data.loc[:,['Project Essay','Project Title']]

tweets_df['Project Essay length'] = Data['Project Essay'].apply(len)
tweets_df['Project Title length'] = Data['Project Title'].apply(len)


TypeError: object of type 'float' has no len()

11